# Skelettschätzer

In [ ]:
!pip install ipywidgets
!pip install tomli
!jupyter labextension install @jupyter-widgets/jupyterlab-manager

import os
import ipywidgets as widgets
from pathlib import Path
from IPython.display import display, clear_output, HTML

## 1. Auswählen von Videos
Wählen Sie zunächst Videos aus, die verarbeitet werden. Sie können mp4-Dateien ins Arbeitsverzeichnis laden. Außerdem können Sie YouTube-Videos per Link einfügen, die dann verarbeitet werden.

In [ ]:
# Data-Ordner finden
with open("../.env", "r") as f:
    env = [line.replace("\n", "").split("=") for line in f.readlines()]
env = {e[0]: e[1] for e in env}
data_folder = env["INPUT_FOLDER"]
input_folder = os.path.join(data_folder, "input")
input_folder_parsed = Path(input_folder)
# YouTube-Videos suchen
if "input.txt" in os.listdir(input_folder):
    with open(os.path.join(input_folder, "input.txt"), "r") as f:
        yt_links = f.read()
else:
    yt_links = ""

# ---------- Widgets ----------

# Datei-Upload (nur mp4 zulassen)
upload_widget = widgets.FileUpload(
  accept='.mp4',
  multiple=True,
  description='MP4 hochladen'
)

# Mehrzeiliges Eingabefeld für URLs
url_textarea = widgets.Textarea(
  placeholder='Gib hier YouTube-Links ein (einer pro Zeile)',
  description='URLs:',
  layout=widgets.Layout(width='100%', height='100px'),
  value=yt_links
)

# Button zum Löschen aller MP4-Dateien
delete_button = widgets.Button(
  description='Alle MP4-Dateien löschen',
  button_style='danger'
)

# Bestätigungsbutton
confirm_button = widgets.Button(
  description='Auswahl bestätigen',
  button_style='success'
)

# Ausgabe für die Dateiliste
file_list_output = widgets.Output()

# ---------- Funktionen ----------

def save_uploaded_files(change):
    for fileinfo in upload_widget.value:
        file_path = input_folder_parsed / fileinfo["name"]
        with open(file_path, 'wb') as f:
            f.write(fileinfo['content'])
    upload_widget.value = []
    update_file_list()

def update_file_list():
  with file_list_output:
    clear_output()
    mp4_files = list(input_folder_parsed.glob('*.mp4'))
    if mp4_files:
      display(HTML('<b>Vorhandene MP4-Dateien:</b>'))
      for file in mp4_files:
        display(HTML(f'📁 {file.name}'))
    else:
      display(HTML('<i>Keine MP4-Dateien im Ordner.</i>'))

def delete_all_mp4_files(b):
  for file in input_folder_parsed.glob('*.mp4'):
    file.unlink()
  update_file_list()

def confirm_selection(b):
  urls = url_textarea.value.strip().splitlines()
  print('✅ Bestätigt:')
  print(f'- {len(list(input_folder_parsed.glob("*.mp4")))} MP4-Dateien im Ordner')
  print(f'- {len(urls)} URLs eingegeben')
  for url in urls:
    print(f'  • {url.strip()}')
  with open(input_folder_parsed / 'input.txt', 'w') as f:
      f.write("\n".join(urls))

# ---------- Event-Handler verbinden ----------

upload_widget.observe(save_uploaded_files, names='value')
delete_button.on_click(delete_all_mp4_files)
confirm_button.on_click(confirm_selection)

# ---------- UI anzeigen ----------

display(widgets.VBox([
  upload_widget,
  url_textarea,
  delete_button,
  confirm_button,
  file_list_output
]))

# Initiale Anzeige
update_file_list()

## 2. Ausführen der Skelettschätzung
Die Skelettschätzungen laufen auf Docker-Containern. Mit einem Klick auf den Button starten Sie die Ausführung.

⚠ Das kann eine Weile dauern.

In [ ]:
# Widgets
status_output = widgets.Output()

def confirm(b):
    with status_output:
        clear_output()
        display(HTML('<p style="color:orange;font-weight:bold;">⏳ Programm wird ausgeführt ...</p>'))
    !docker compose -f ../docker-compose.yml up
    with status_output:
        clear_output(wait=True)
        display(HTML('<p style="color:darkgreen;font-weight:bold;">✅ Ausführung beendet</p>'))
    try:
        update_dropdown()
    except NameError:
        pass

# Button
confirm_button = widgets.Button(
  description='Programm starten',
  button_style='success'
)
confirm_button.on_click(confirm)

# Anzeige
display(widgets.VBox([confirm_button, status_output]))

Statusmeldungen während der Ausführung der Container können Sie sich in den Logs ansehen:
`View > Show Log Console`

## 3. Auswählen des auszuwertenden Videos
Bitte wählen Sie, welches Video Sie auswerten möchten.

In [ ]:
def get_output_folders():
    return [os.path.join(data_folder, dir) for dir in os.listdir(data_folder) if dir != "input"]

def get_output_videos():
    filelists = [[file for file in os.listdir(dir) if file.endswith(".mp4")] for dir in get_output_folders()]
    filelist = set(filelists[0])
    for f in filelists[1:]:
        filelist &= set(f)
    return filelist

def update_dropdown():
    video_dropdown.options = get_output_videos()

video_dropdown = widgets.Dropdown(
    options=get_output_videos(),
    description='Video:',
    disabled=False
)

# Button
video_commit_button = widgets.Button(
    description='Auswählen',
    button_style='success'
)

# Funktion, die beim Klick auf den Button ausgeführt wird
def commit_video(b):
    print(f'Du hast "{dropdown.value}" ausgewählt.')

# Button mit Funktion verbinden
video_commit_button.on_click(commit_video)

# Widgets anzeigen
display(video_dropdown, video_commit_button)
